In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 53kB/s 
     |████████████████████████████████| 204kB 44.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=a07e3251b639f1580498f4fa22864a94ab45ebc516ec66b78195e5ab57c73c64
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark import SparkContext
import xgboost as xgb
sc=SparkContext.getOrCreate()
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error as mse
import numpy as np 
import glob 
import csv
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [6]:
dataset=input('Wich Dataset(1,2,3 or 4): ').upper()
rep=input('Wich rep (F or P): ').upper()
if rep=='F': 
    rep='Frequency' 
elif rep=='P':
    rep='Presence'
else: 
    exit()
path='./drive/My Drive/sample_data/XGB/'+dataset+'/'+rep+'/*.csv'
files = glob.glob(path)
dic={}
for n in files:
	x=n.split('/')
	id=x[len(x)-1].split('.')
	dic[int(id[0])]= n
sorted_dict = {k: dic[k] for k in sorted(dic)}


Wich Dataset(1,2,3 or 4): 2
Wich rep (F or P): p


In [0]:
print(len(sorted_dict))

21


In [7]:
list_accuracy=[]
params = {'objective': 'reg:squarederror', 'verbose': False}
suma=0
for k,n in sorted_dict.items():
  p=n.replace('\\','/')
  # load data
  win=pd.read_csv(p,header=None)
  win = win.sample(frac=1).reset_index(drop=True)

  num_col_chunk_train=len(win.columns)-1
  # split data into X and y
  X_train_1= win.loc[:,0:(num_col_chunk_train-1)]
  y_train_1= win.loc[:,num_col_chunk_train] 
  # CV model  
  model = xgb.XGBClassifier()
  kfold = KFold(n_splits=10, random_state=7)
  '''
  f you have many classes for a classification type predictive modeling problem or the 
  classes are imbalanced (there are a lot more instances for one class than another), 
  it can be a good idea to create stratified folds when performing cross validation.
  This has the effect of enforcing the same distribution of classes in each fold as in 
  the whole training dataset when performing the cross validation evaluation. The scikit-learn library provides 
  this capability in the StratifiedKFold class.
  '''
  #kfold = StratifiedKFold(n_splits=10, random_state=7)
  results = cross_val_score(model, X_train_1, y_train_1, cv=kfold)
  res_mean=results.mean()*100
  print("Accuracy: %.2f%% (%.2f%%)" % (res_mean, results.std()*100))
  suma=suma+res_mean
av=suma/len(sorted_dict)
print('The average accuracy is '+str(av))

Accuracy: 75.50% (25.04%)
Accuracy: 88.00% (18.33%)
Accuracy: 82.58% (9.21%)
Accuracy: 87.86% (4.63%)
Accuracy: 83.00% (11.11%)
Accuracy: 79.52% (11.87%)
Accuracy: 78.67% (13.18%)
Accuracy: 85.53% (14.13%)
Accuracy: 86.52% (10.72%)
Accuracy: 85.98% (4.34%)
Accuracy: 77.33% (16.53%)
Accuracy: 86.39% (9.46%)
Accuracy: 82.45% (13.37%)
Accuracy: 87.39% (6.86%)
Accuracy: 90.16% (7.36%)
Accuracy: 87.83% (6.30%)
The average accuracy is 84.04550214554011
